In [ ]:
import os
import shutil
import tempfile
import numpy as np
from itertools import combinations

import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from monai import transforms
from monai.apps import DecathlonDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.utils import first, set_determinism
from torch.cuda.amp import GradScaler, autocast
from torch.nn import L1Loss
from tqdm import tqdm

from monai.transforms import (
    EnsureChannelFirstd,
    CenterSpatialCropd,
    Compose,
    Lambdad,
    LoadImaged,
    Resized,
    ScaleIntensityd,
)
from monai.utils import set_determinism
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

from generative.inferers import DiffusionInferer
from generative.networks.nets import DiffusionModelUNet
from generative.networks.schedulers import DDPMScheduler, DDIMScheduler
from monai.data import (
    CacheDataset,
    Dataset,
    DataLoader,
    LMDBDataset,
    PersistentDataset,
    decollate_batch,
)

from generative.inferers import LatentDiffusionInferer
from generative.losses import PatchAdversarialLoss, PerceptualLoss
from generative.networks.nets import AutoencoderKL, DiffusionModelUNet, PatchDiscriminator
from generative.networks.schedulers import DDPMScheduler
from generative.metrics import FIDMetric, MMDMetric, MultiScaleSSIMMetric, SSIMMetric

GPU_NUM = '4'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NUM
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
print("Current GPU:",  os.environ["CUDA_VISIBLE_DEVICES"])

device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu") #ADDED LATER

ms_ssim = MultiScaleSSIMMetric(spatial_dims=3, data_range=1.0, kernel_size=2)
ssim = SSIMMetric(spatial_dims=3, data_range=1.0, kernel_size=2)

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".npy"):
            img_path = os.path.join(folder, filename)
            img = np.load(img_path)
            images.append(img)
    return images

male_images = load_images_from_folder('../cjagad/LDM/synthetic_images/male/DDPM_CONDITIONAL_MALE_3000')
print(len(male_images))

male_images = [torch.tensor(img) for img in male_images]

male_images_tensor = torch.stack(male_images)

ms_ssim_scores = []
ssim_scores = []

idx_pairs = list(combinations(range(male_images_tensor.shape[0]), 2))
print(len(idx_pairs))

print("CONDITONAL FREE FEMALE 500 - Diversity (Synthetic vs Synthetic) MS-SSIM AE-80 LDM-2000")

for idx_a, idx_b in tqdm(idx_pairs, desc="Calculating SSIM and MS-SSIM", unit="pair"):
    img_a = male_images_tensor[[idx_a]].squeeze().unsqueeze(0).unsqueeze(1)  
    img_b = male_images_tensor[idx_b].squeeze().unsqueeze(0).unsqueeze(1)    
    ms_ssim_scores.append(ms_ssim(img_a, img_b))
    ssim_scores.append(ssim(img_a, img_b))

ms_ssim_scores = torch.cat(ms_ssim_scores, dim=0)
ssim_scores = torch.cat(ssim_scores, dim=0)
print(f"MS-SSIM Metric: {ms_ssim_scores.mean():.4f} +- {ms_ssim_scores.std():.4f}")
print(f"SSIM Metric: {ssim_scores.mean():.4f} +- {ssim_scores.std():.4f}")

Current GPU: 4
500
124750
CONDITONAL FREE FEMALE 500 - Diversity (Synthetic vs Synthetic) MS-SSIM AE-80 LDM-2000


Calculating SSIM and MS-SSIM:  95%|██▊| 118436/124750 [31:10<02:59, 35.11pair/s]